# Описание
Здесь представлена модель для [соревнования](https://stepik.org/lesson/226979/step/1?unit=199528), 
который дается в конце курса ["Введение в Data Science и машинное обучение"](https://stepik.org/course/4852/) на stepic.

ROC score на сабмите 0.8933.
Занял 3 место.

Использованы признаки модели model-auto_features.ipynb и признаки сгенерированные вручную(совсем немного).

# Импорт

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook
import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.externals import joblib

sys.path.append(os.path.join(sys.path[0], '../'))

import libs.config as conf
import libs.data_helpers as dh
import libs.data_iter_final as di
import libs.utils.model_utils as mu
import libs.submit_report as rep

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/home/jupyter/ve/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Загрузка и подготовка данных

In [2]:
# загрузка данных
events  = pd.read_csv(f"{conf.DATA_DIR}/event_data_train.zip")
submissions = pd.read_csv(f"{conf.DATA_DIR}/submissions_data_train.zip")

# генерация признаков
X_cv, y_cv = di.get_x_y(events, submissions)
print ('X_cv shape', X_cv.shape)
y_cv.value_counts(dropna=False)

/home/jupyter/ve/lib/python3.6/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  
/home/jupyter/ve/lib/python3.6/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (413) have mixed types. Specify dtype option on import or set low_memory=False.
  


X_cv shape (19234, 387)


False    17266
True      1968
Name: is_gone, dtype: int64

In [3]:
# разделение данных для обучения на train и test
X_train, X_test, y_train, y_test = train_test_split(X_cv, y_cv, test_size=0.2, stratify=y_cv)
print ('X_train shape', X_train.shape)
y_train.value_counts(dropna=False)

X_train shape (15387, 387)


False    13813
True      1574
Name: is_gone, dtype: int64

# Обучение и проверка модели

## RandomForest

In [4]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=2, 
                            min_samples_leaf=10, min_samples_split=10, 
                            class_weight='balanced')

rf.fit(X_train, y_train)
pred_proba = rf.predict_proba(X_test)
roc_score = roc_auc_score(y_test, pred_proba[:, 1])
print('roc на test', roc_score)
# должны получить на тест roc 0.907  +- 0.02

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=10,
                       min_samples_split=10, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=2, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

roc на test 0.9043574262651032


In [5]:
# важность фич
fimp = mu.get_feature_importances_df(rf.feature_importances_, X_train.columns)
fimp.head(15)

,weight
correct_rat_discovered,0.044796
correct_rat_passed,0.041764
correct__mean,0.041464
correct_rat_day,0.032540
correct_rat_started_attempt,0.031382
correct_rat_viewed,0.028542
correct__length,0.024345
viewed__length,0.021888
viewed__variance,0.020518
discovered__length,0.018706


## кроссвалидация

In [6]:
# значение к метрике на кроссвалидации коррелирует к метрике на степике

rfcv = RandomForestClassifier(**rf.get_params())

cv_scores = cross_val_score(rfcv, X_cv, y_cv, scoring='roc_auc', cv=10, n_jobs=-1)
mean_cv_scores = np.mean(cv_scores)
print ('mean score', mean_cv_scores)
# должны получить срдений score 0.902 +- 0.01

mean score 0.9032016905897304


## Сохранение модели

In [7]:
# сохранить модель
!mkdir $conf.BIN_MODELS_DIR
model_fname = f"{conf.BIN_MODELS_DIR}/final_model.bin"
joblib.dump(rf, model_fname)

# загрузка модели
# rf = joblib.load(model_fname)

mkdir: cannot create directory ‘../bin_models’: File exists


['../bin_models/final_model.bin']

# Сделать предсказание по сабмиту

In [8]:
SUBMIT_NUM = 9

events_pred  = pd.read_csv(f"{conf.DATA_DIR}/events_data_test.zip")
submissions_pred = pd.read_csv(f"{conf.DATA_DIR}/submission_data_test.zip")
X_pred , _ = di.get_x_y(events_pred, submissions_pred)

pred_proba = rf.predict_proba(X_pred)[:, 1]
rep_df = rep.create_report(X_pred.index, pred_proba)
assert rep_df.user_id.nunique() == X_pred.index.nunique()
print ('Прогноз сохранен в файл ', rep.save_report(rep_df, SUBMIT_NUM))

print ('Распределение "вероятностей" модели')
pd.cut(pred_proba, 10).value_counts()

/home/jupyter/ve/lib/python3.6/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  """
/home/jupyter/ve/lib/python3.6/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (413) have mixed types. Specify dtype option on import or set low_memory=False.
  """


Прогноз сохранен в файл  ../reports/predict_2019-06-06_submit_9.csv
Распределение "вероятностей" модели


(0.00729, 0.104]    3451
(0.104, 0.2]         311
(0.2, 0.296]         682
(0.296, 0.392]       636
(0.392, 0.488]       362
(0.488, 0.584]       229
(0.584, 0.68]        172
(0.68, 0.776]        150
(0.776, 0.872]       114
(0.872, 0.968]        77
dtype: int64